**BERT 모델 불러오기**

In [ ]:
from transformers import BertModel, BertTokenizer
import torch

# 1. BERT 토크나이저와 모델을 로드
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

In [ ]:
# tokenizer의 내장함수를 이용해 특수 토큰을 불러오는 함수를 작성해주세요
tokenizer.get_special_tokens_map


{'unk_token': '[UNK]',
 'sep_token': '[SEP]',
 'pad_token': '[PAD]',
 'cls_token': '[CLS]',
 'mask_token': '[MASK]'}

**1.BERT Tokenizer의 특수 토큰**

아래 코드에서 특수 토큰을 불러오는 함수를 작성하고 각 토큰이 무엇을 의미하는지 적어주세요

- [UNK] : unknown token, 사전에 없는 단어를 처리할 때 사용. 인식하지 못하는 단어를 UNK로 사용
- [SEP] : Seperate token, 문장 간 구분을 의미
- [PAD] : batch 내 서로 다른 길이 문장을 동일한 길이로 맞출 때 사용
- [CLS] : 문장 시작 토큰
- [MASK] : 마스킹된 단어 토큰

**tokenizer를 이용해 문장을 토큰화**

토큰화를 통해 문자를 숫자로 변환하여 컴퓨터가 이해할 수 있도록 한다.

In [ ]:
# 입력 문장
sentence = "Hello, this is a sentence for tokenization."

# 문장 토큰화 및
inputs = tokenizer(sentence, return_tensors="pt")

# 토큰 ID를 다시 토큰으로 변환
tokens = tokenizer.convert_ids_to_tokens(inputs['input_ids'][0].tolist())  # 텐서를 리스트로 변환

print("Token IDs:", inputs['input_ids'])
print("Tokens:", tokens)

Token IDs: tensor([[  101,  7592,  1010,  2023,  2003,  1037,  6251,  2005, 19204,  3989,
          1012,   102]])
Tokens: ['[CLS]', 'hello', ',', 'this', 'is', 'a', 'sentence', 'for', 'token', '##ization', '.', '[SEP]']


**2. 토큰화**
토큰화 이후에 [CLS] 토큰과 [SEP] 토큰이 추가된 이유를 작성하세요

- [CLS] 토큰이 추가된 이유 : 문장 전체를 대표하는 임베딩을 제공할 수 있음
- [SEP] 토큰이 추가된 이유 : 여러문장을 하나의 입력을 넣을 때 사용

**감성 분류를 위한 사전 학습된 BERT 모델을 이용해 감성분류하기**

In [3]:
import torch
from transformers import BertForSequenceClassification
from torch.nn.functional import softmax

# 감성 분류를 위한 사전 학습된 모델
model_name = "nlptown/bert-base-multilingual-uncased-sentiment"
model = BertForSequenceClassification.from_pretrained(model_name)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/953 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/669M [00:00<?, ?B/s]

In [6]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch.nn.functional as F

def sentiment_analysis(sentence):
    model_name = "nlptown/bert-base-multilingual-uncased-sentiment"
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = BertForSequenceClassification.from_pretrained(model_name)

    inputs = tokenizer(sentence, return_tensors="pt", truncation=True, padding=True)

    with torch.no_grad():
        outputs = model(**inputs)

    probs = F.softmax(outputs.logits, dim=-1)

    probs_list = probs[0].tolist()

    class_label = ['매우 부정적', '부정적', '중립적', '긍정적', '매우 긍정적']

    for i, prob in enumerate(probs_list):
        print(f"{class_label[i]}일 확률: {prob:.2f}")

**3. BERT를 이용하여 각각 class에 맞는 sentence 작성해보기**

1. 매우 부정적인 문장을 작성해서 매우 부정적에 대한 확률을 최대화 해보세요
2. 부정적인 문장을 작성해서 매우 부정적에 대한 확률을 최대화 해보세요
3. 중립적인 문장을 작성해서 매우 부정적에 대한 확률을 최대화 해보세요
4. 긍정적인 문장을 작성해서 매우 부정적에 대한 확률을 최대화 해보세요
5. 매우 긍정적인 문장을 작성해서 매우 부정적에 대한 확률을 최대화 해보세요

In [7]:
# 매우 부정적인 문장 작성
sentence = "나는 바보 천치 똥멍청이야"

# 감성분석 수행
sentiment_analysis(sentence)


매우 부정적일 확률: 0.38
부정적일 확률: 0.30
중립적일 확률: 0.18
긍정적일 확률: 0.07
매우 긍정적일 확률: 0.07


In [8]:
# 부정적인 문장 작성
sentence = "나는 적당한 바보 해삼 말미잘이야"

# 감성분석 수행
sentiment_analysis(sentence)

매우 부정적일 확률: 0.46
부정적일 확률: 0.31
중립적일 확률: 0.15
긍정적일 확률: 0.05
매우 긍정적일 확률: 0.04


In [9]:
# 중립적 문장 작성
sentence = "나는 공평한 사람이야"

# 감성분석 수행
sentiment_analysis(sentence)

매우 부정적일 확률: 0.36
부정적일 확률: 0.20
중립적일 확률: 0.13
긍정적일 확률: 0.10
매우 긍정적일 확률: 0.21


In [10]:
# 긍정적 문장 작성
sentence = "사랑해"

# 감성분석 수행
sentiment_analysis(sentence)

매우 부정적일 확률: 0.02
부정적일 확률: 0.02
중립적일 확률: 0.07
긍정적일 확률: 0.21
매우 긍정적일 확률: 0.68


In [11]:
# 매우 긍정적인 문장 작성
sentence = "매우 사랑해"

# 감성분석 수행
sentiment_analysis(sentence)

매우 부정적일 확률: 0.01
부정적일 확률: 0.01
중립적일 확률: 0.03
긍정적일 확률: 0.17
매우 긍정적일 확률: 0.77


**4. BERT의 감성 분류 기준에 대해 생각을 적어보기**


BERT가 어떤 요소에 따라 감성 분류 기준을 나누는지에 대해 적어보세요

내가 생각하는 분류 기준 : 단어 의미에 따라 긍 부정을 나누는 것 같고, 접두사에 의해 강조되는것같음